## Import Dependencies

In [1]:
import datetime
import hashlib

## Create a Block

In [2]:
#define the 'block' data structure
class Block:
    # in this case, each block has 7 attr
    
    block_no = 0
    data = None
    next = None
    hash = None
    nounce = 0
    prev_hash = 0x0
    timestamp = datetime.datetime.now()
    
    def __init__(self, data):
        self.data = data
        
    def hash(self):
        h = hashlib.sha256()
        h.update(
            str(self.nounce).encode('utf-8') +
            str(self.data).encode('utf-8') +
            str(self.prev_hash).encode('utf-8') +
            str(self.timestamp).encode('utf-8') +
            str(self.block_no).encode('utf-8')
        )
        return h.hexdigest()
    
    def __str__(self):
        return "Block Hash: " + str(self.hash()) + "\n Prev Hash: "+ str(self.prev_hash) +"\nBlockNo: "  + str(self.block_no) + "\nBlock Data: " + str(self.data) + "\nHashes: " + str(self.nounce) + "\n----------------"

## Create a Blockchain

In [3]:
class Blockchain:
    
    # sets the difficulty by shrinking the accepted target
    diff = 20
    max_nounce = 2**32 # 32bit max
    target = 2 ** (256-diff) # target hash, for mining
    
    # generate the 'genesis' (first) block
    block = Block("Genesis")
    dummy = head = block
    
    def add(self, block):
        # set the hash of current block to prev
        block.prev_hash = self.block.hash()
        block.block_no = self.block.block_no +1
        
        self.block.next = block
        self.block = self.block.next
        
    def mine(self, block):
        #from 0 to 2^32
        for n in range(self.max_nounce):
            #..is the value less than our target?
            
            if int(block.hash(), 16) <= self.target:
                # ..if it is add to chain
                self.add(block)
                print(block)
                break
            else:
                block.nounce += 1

In [4]:
blockchain = Blockchain()
andreas_block = Block("Andreas L")

In [5]:
blockchain.mine(andreas_block)

Block Hash: 8b82ed4edc09dbea6df26ca96b23dcda2eaf2d58a3c2760db65ff40ef0441f53
 Prev Hash: 20575689cb5e9b143fecc060a9134e146fe5da378d36ae5024b36964d0beb94e
BlockNo: 1
Block Data: Andreas L
Hashes: 389886
----------------


In [6]:
blockchain.mine(Block("Therese"))

Block Hash: 0646d91d55c3fe7526725b694a58958a8106c213ceda452be6147c039b7a6e99
 Prev Hash: 8b82ed4edc09dbea6df26ca96b23dcda2eaf2d58a3c2760db65ff40ef0441f53
BlockNo: 2
Block Data: Therese
Hashes: 897918
----------------


In [7]:
while blockchain.dummy != None:
    print(blockchain.dummy)
    blockchain.dummy = blockchain.dummy.next

Block Hash: 20575689cb5e9b143fecc060a9134e146fe5da378d36ae5024b36964d0beb94e
 Prev Hash: 0
BlockNo: 0
Block Data: Genesis
Hashes: 0
----------------
Block Hash: 8b82ed4edc09dbea6df26ca96b23dcda2eaf2d58a3c2760db65ff40ef0441f53
 Prev Hash: 20575689cb5e9b143fecc060a9134e146fe5da378d36ae5024b36964d0beb94e
BlockNo: 1
Block Data: Andreas L
Hashes: 389886
----------------
Block Hash: 0646d91d55c3fe7526725b694a58958a8106c213ceda452be6147c039b7a6e99
 Prev Hash: 8b82ed4edc09dbea6df26ca96b23dcda2eaf2d58a3c2760db65ff40ef0441f53
BlockNo: 2
Block Data: Therese
Hashes: 897918
----------------


### Ev0l Haxx0r changing middle block data

In [8]:
blockchain.dummy = blockchain.head
while blockchain.dummy != None:
    # ev0l haxxor ...
    if(blockchain.dummy.data == 'Andreas L'):
        blockchain.dummy.data = 'Murkel-olle'
    
    print(blockchain.dummy)
    blockchain.dummy = blockchain.dummy.next

Block Hash: 20575689cb5e9b143fecc060a9134e146fe5da378d36ae5024b36964d0beb94e
 Prev Hash: 0
BlockNo: 0
Block Data: Genesis
Hashes: 0
----------------
Block Hash: 5c862ac9c68e576883d757bc114841ee5623188bfcf762df63a3e45da3053e13
 Prev Hash: 20575689cb5e9b143fecc060a9134e146fe5da378d36ae5024b36964d0beb94e
BlockNo: 1
Block Data: Murkel-olle
Hashes: 389886
----------------
Block Hash: 0646d91d55c3fe7526725b694a58958a8106c213ceda452be6147c039b7a6e99
 Prev Hash: 8b82ed4edc09dbea6df26ca96b23dcda2eaf2d58a3c2760db65ff40ef0441f53
BlockNo: 2
Block Data: Therese
Hashes: 897918
----------------


### Verify

In [9]:
blockchain.dummy = blockchain.head
while blockchain.dummy != None:
    if blockchain.dummy.next and blockchain.dummy.prev_hash != 0:
        if blockchain.dummy.next.prev_hash != blockchain.dummy.hash():
            print("Inconsistency at "+blockchain.dummy.data)
    blockchain.dummy = blockchain.dummy.next

Inconsistency at Murkel-olle


### Restore

In [10]:
blockchain.dummy = blockchain.head
while blockchain.dummy != None:
    # restore to original data ...
    if(blockchain.dummy.data == 'Murkel-olle'):
        blockchain.dummy.data = 'Andreas L'
    
    print(blockchain.dummy)
    blockchain.dummy = blockchain.dummy.next

Block Hash: 20575689cb5e9b143fecc060a9134e146fe5da378d36ae5024b36964d0beb94e
 Prev Hash: 0
BlockNo: 0
Block Data: Genesis
Hashes: 0
----------------
Block Hash: 8b82ed4edc09dbea6df26ca96b23dcda2eaf2d58a3c2760db65ff40ef0441f53
 Prev Hash: 20575689cb5e9b143fecc060a9134e146fe5da378d36ae5024b36964d0beb94e
BlockNo: 1
Block Data: Andreas L
Hashes: 389886
----------------
Block Hash: 0646d91d55c3fe7526725b694a58958a8106c213ceda452be6147c039b7a6e99
 Prev Hash: 8b82ed4edc09dbea6df26ca96b23dcda2eaf2d58a3c2760db65ff40ef0441f53
BlockNo: 2
Block Data: Therese
Hashes: 897918
----------------


### Verify

In [11]:
blockchain.dummy = blockchain.head
while blockchain.dummy != None:
    if blockchain.dummy.next and blockchain.dummy.prev_hash != 0:
        if blockchain.dummy.next.prev_hash != blockchain.dummy.hash():
            print("Inconsistency at "+blockchain.dummy.data)
    blockchain.dummy = blockchain.dummy.next

wheee! 